# Jonathan Halverson
# Friday, May 6, 2016
# Accumulators and broadcast variables

### Accumulators

Accumulators are counters that are passed to each worker node where they may be incremented. The sum of the local values is made available in the driver:

In [1]:
lines = sc.textFile('text_file.md')
print lines.first()
print lines.count()

# Apache Spark
95


In [2]:
blankLines = sc.accumulator(0)

In [3]:
def extractWords(line):
    global blankLines
    print line
    if line == '':
        blankLines += 1
    return line.split()

In [4]:
words = lines.flatMap(extractWords)
words.count()
print 'Blank lines:', blankLines.value

Blank lines: 35


Note that we need to call count in order to get past the lazy evaluation of Spark. We could write the result to file to force the evaluation. The updating of an accumulator is a side-effect.

In [5]:
dir(blankLines)

['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_deserialized',
 '_value',
 'accum_param',
 'add',
 'aid',
 'value']

### Broadcast variables

Broadcast variables are read-only and are good for sending large data structures over the network such as a lookup table or the like.

In [6]:
large_list = sc.broadcast(range(0, 100000, 10))
print type(large_list)

<class 'pyspark.broadcast.Broadcast'>


In [7]:
def filterBasedOnBroadcast(x):
    return x in large_list.value

In [8]:
new_RDD = lines.map(len).filter(filterBasedOnBroadcast)
print new_RDD.collect()

[0, 0, 0, 0, 0, 70, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 120, 0, 0, 0, 120, 0, 0, 70]


We see that indeed only the multiples of 10 remain after the filter is applied.